In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import copy
from copy import deepcopy
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# import torch
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import torch
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import json
import chardet
from transformers import BertConfig,BertTokenizer,BertTokenizerFast

In [ ]:

datas_type=[]
datas=[]
file=open("/kaggle/input/fewfc/train_label.json", "rb")
for line in file:
    datas.append(json.loads(line))
file.close()
texts=[]

for data in datas:
    if data['content'][-1].isalnum()==False:
        data['content']=data['content'][:-1]
    texts.append(data['content'])

In [ ]:
def FCFew_data_label(data,token_span,ovlp_id):
    result_span = torch.zeros(len(token_span),len(token_span))
    result_relation = torch.zeros(len(token_span),len(token_span))
    overlap_ids=torch.zeros(len(token_span))
    index=data['index']
    trigger_start=-1
    d=data['triggers'][index]
    tri_start,tri_end=d[0],d[1]
    arg_list=[]
    arg_start,arg_end=-1,-1
    for i, (offset_start, offset_end) in enumerate(token_span):
        if offset_start >= tri_start and offset_end <= tri_end:
            if offset_start == tri_start:
                trigger_start=i
            if offset_end == tri_end:
                trigger_end=i
                result_span[trigger_start,trigger_end]=2 #触发词
                for pos in range(trigger_start,trigger_end+1):
                    overlap_ids[pos]=ovlp_id
                continue
        for value in data['args'].values():
            flag=0
            for v in value:
                if v[0] <= offset_start and v[1] >= offset_end:
                    if offset_start == v[0]:
                        arg_start=i
                        flag=1
                        if offset_end == v[1]:
                            result_span[i,i]=1
                        break
                        
                    if offset_end == v[1]:
                        arg_end=i
                        result_span[arg_start,arg_end]=1 #论元
                        arg_list.append([arg_start,arg_end])
                        flag=1
                        break

            if flag==1:
                break

    assert (overlap_ids>=1).sum()!=0,print('error:',data)
    assert trigger_start!=-1
    for s,e in arg_list:
            if result_relation[trigger_start][s]==0 and result_relation[trigger_start][e]==0:
                result_relation[trigger_start][s]=1
                result_relation[trigger_start][e]=1 #触发词没有重叠

            else:
                if trigger_start!=trigger_end and result_relation[trigger_start][s]!=0 and result_relation[trigger_start][e]!=0:
                    assert 1==0,print(data['id'],arg_list)
                    result_relation[trigger_start][s]=2
                    result_relation[trigger_start][e]=2 #触发词没有重叠
            if result_relation[trigger_end][s]==0 and result_relation[trigger_end][e]==0:
                result_relation[trigger_end][s]=1
                result_relation[trigger_end][e]=1 #触发词没有重叠

            else:
                if trigger_start!=trigger_end and result_relation[trigger_end][s]!=0 and result_relation[trigger_end][e]!=0:
                    print(data['id'],arg_list)
                    assert 1==0
                    result_relation[trigger_end][s]=2
                    result_relation[trigger_end][e]=2 #触发词没有重叠

    return result_span,result_relation,overlap_ids      

In [ ]:
def label_sparse(labels):
    indices = torch.nonzero(labels)
    values = labels[indices[:,0],indices[:,1]]
    sparse_labels = torch.sparse_coo_tensor(indices.t(), values, size=labels.shape)
    return sparse_labels

In [ ]:
def main():
    multilingual_tokenizer_fast=BertTokenizerFast.from_pretrained('bert-base-multilingual-cased')
    new_datas=[] #保存添加label的json数据
    add_datas=[] #保存修改后的json数据
    temp_index=[]
    pre_id=None
    triggers_count={}
    for i,text in enumerate(texts):
        j=0
        token_results=multilingual_tokenizer_fast.encode_plus(text,return_offsets_mapping=True,add_special_tokens=False)
        #判断文本的长度,超过则删除数据
        token_len=len(token_results['input_ids'])
        if token_len>126:
            continue
        #判断触发词的重叠次数,超过则删除数据
        flag=0


        token_span=token_results['offset_mapping']
        
        if j<len(datas_type) and datas[i]['id']!=datas[j]['id']:
            temp_index=[]
            j=i
        
    #     if j>=len(datas_type):
    #         j=0
    #         continue
        data={}
        data['id']=datas[i]['id']
        data['content']=datas[i]['content']
        data['triggers']=datas[i]['triggers']
        data['index']=datas[i]['index']
        data['args']=datas[i]['args']
        
        if pre_id==None:
            pre_id=data['id']
    #         pre_tri=data['triggers']
    #         pre_index=data['index']
            triggers_count[data['trigger'][data['index']]]=1
        else:
            if pre_id != data['id']:
                for k,v in triggers_count:
                    if v<=1:
                        
                temp_data=deepcopy(new_datas[-1])
                size=temp_data['span_label'][2]
                temp_data['span_label']=[]
                temp_data['span_label'].append(size)
    #             size=data['relation_label'][2]
                temp_data['relation_label']=[]
                temp_data['relation_label'].append(size)
                for i,d in enumerate(temp_data['overlap_ids']):
                    if d==1:
                        temp_data['overlap_ids'][i]=2.0
                pre_id = data['id']
                new_datas.append(temp_data)
        index_count=0
        for index in temp_index:
            if index==data['triggers'][data['index']]:
                index_count=index_count+1
        temp_index.append(data['triggers'][data['index']])  
        ovlp_id=index_count+1
        span_label,relation_label,overlap_ids=FCFew_data_label1(data,token_span,ovlp_id)

        #转换成稀疏标签
        span_label=label_sparse(span_label)
        relation_label=label_sparse(relation_label)

        a=[span_label._indices().tolist(),span_label._values().tolist(),span_label.size()]
    #     datas[i]['span_label']=[{k:v} for k,v in zip(['indices','values','size'],a)]
        data['span_label']=a
        b=[relation_label._indices().tolist(),relation_label._values().tolist(),relation_label.size()]
        data['relation_label']=b

        data['overlap_ids']=overlap_ids.tolist()


        #添加修改contexnt后的json数据
        new_datas.append(data)

    with open('/kaggle/working/new_train_labels.json','a') as out_file:
        for data in new_datas:
            json.dump(data,out_file,ensure_ascii=False)
            out_file.write('\n')

if __name__ == "__main__":
    main()